# CleanUp


## Prerequisites

Select the `Python 3 (Data Science)` kernel in SageMaker Studio.

In [ ]:
%store -r
%store

In [15]:
import boto3
import sagemaker

region = boto3.Session().region_name
print(f"Region: {region}")
role = sagemaker.get_execution_role()
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client("sagemaker")
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client)
bucket = sagemaker_session.default_bucket()
print(f"Bucket: {bucket}")

Region: us-east-1
Bucket: sagemaker-us-east-1-875692608981


In [4]:
from datetime import datetime
from sagemaker.lineage.context import Context
from sagemaker.lineage.action import Action
from sagemaker.lineage.association import Association
from sagemaker.lineage.artifact import Artifact

## Cleanup

In [ ]:
import time
def delete_associations(arn):
    # delete incoming associations
    incoming_associations = Association.list(destination_arn=arn)
    for summary in incoming_associations:
        assct = Association(
            source_arn=summary.source_arn,
            destination_arn=summary.destination_arn,
            sagemaker_session=sagemaker_session,
        )
        assct.delete()
        

    # delete outgoing associations
    outgoing_associations = Association.list(source_arn=arn)
    for summary in outgoing_associations:
        assct = Association(
            source_arn=summary.source_arn,
            destination_arn=summary.destination_arn,
            sagemaker_session=sagemaker_session,
        )
        assct.delete()


def delete_lineage_data():
    for summary in Context.list():
        print(f"Deleting context {summary.context_name}")
        delete_associations(summary.context_arn)
        ctx = Context(context_name=summary.context_name, sagemaker_session=sagemaker_session)
        ctx.delete()
        time.sleep(1)

    for summary in Action.list():
        print(f"Deleting action {summary.action_name}")
        delete_associations(summary.action_arn)
        actn = Action(action_name=summary.action_name, sagemaker_session=sagemaker_session)
        actn.delete()
        time.sleep(1)

    for summary in Artifact.list():
        print(f"Deleting artifact {summary.artifact_arn} {summary.artifact_name}")
        delete_associations(summary.artifact_arn)
        artfct = Artifact(artifact_arn=summary.artifact_arn, sagemaker_session=sagemaker_session)
        artfct.delete()
        time.sleep(1)


delete_lineage_data()

In [ ]:
from model_package_src.demo_helpers import delete_project_resources

In [ ]:

delete_project_resources(
    sagemaker_boto_client=sagemaker_client,
    endpoint_name=endpoint_name, 
    pipeline_name=pipeline_name, 
    mpg_name=mpg_name, 
    prefix=prefix,
    delete_s3_objects=True,
    bucket_name=bucket)


In [ ]:
import sys

!{sys.executable} -m pip install sagemaker-experiments

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

for component in TrialComponent.list():
    print(f"Deleting Trial Component: {component.trial_component_name}")
    comp = TrialComponent(trial_component_name=component.trial_component_name,sagemaker_boto_client=sagemaker_client)
    comp.delete(force_disassociate='true')

for trial in Trial.list():
    print(f"Deleting Trial: {trial.trial_name}")
    t = Trial(trial_name=trial.trial_name,sagemaker_boto_client=sagemaker_client)
    t.delete()    
    
for exp in Experiment.list():
    print(f"Deleting Experiment: {exp.experiment_name}")
    e = Experiment(experiment_name=exp.experiment_name,sagemaker_boto_client=sagemaker_client)
    e.delete()


In [ ]:
endpoints = sagemaker_client.list_endpoints()

for endpoint in endpoints['Endpoints']:
    print(f"Deleting Endpoint: {endpoint['EndpointName']}")
    sagemaker_client.delete_endpoint(
        EndpointName=endpoint['EndpointName']
    )
    
endpoint_configs = sagemaker_client.list_endpoint_configs()

for endpoint_config in endpoint_configs['EndpointConfigs']:
    print(f"Deleting Endpoint Config: {endpoint_config['EndpointConfigName']}")
    sagemaker_client.delete_endpoint_config(
        EndpointConfigName=endpoint_config['EndpointConfigName']
    )

In [ ]:
sagemaker_client.list_endpoint_configs()

In [38]:
response = sagemaker_client.list_monitoring_schedules()
monitorschedulename = response['MonitoringScheduleSummaries'][0]['MonitoringScheduleName']
monitorschedulename
sagemaker_client.delete_monitoring_schedule(
    MonitoringScheduleName=monitorschedulename
)

{'ResponseMetadata': {'RequestId': '7d571eb0-f535-4922-b168-cc3d36b387ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7d571eb0-f535-4922-b168-cc3d36b387ad',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 18 Nov 2021 08:30:41 GMT'},
  'RetryAttempts': 0}}

In [10]:
response = sagemaker_client.list_model_quality_job_definitions()
jobmonitorname = response['JobDefinitionSummaries'][0]['MonitoringJobDefinitionName']
jobmonitorname

'model-quality-job-definition-2021-11-18-06-55-13-446'